In [2]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2023
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [3]:
import mlflow.spark

In [4]:
import os
import warnings
import sys
import mlflow
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [5]:
import logging
import json
import shutil
import datetime
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
import cml.data_v1 as cmldata

In [6]:
import cml.data_v1 as cmldata

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
#SparkContext.setSystemProperty('spark.executor.cores', '1')
#SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "go01-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

# Sample usage to run query through spark
EXAMPLE_SQL_QUERY = "show databases"
spark.sql(EXAMPLE_SQL_QUERY).show()

23/11/05 19:47:22 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/11/05 19:47:22 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 19:47:23 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
23/11/05 

+--------------------+
|           namespace|
+--------------------+
|         01_car_data|
|           01_car_dw|
|              adb101|
|            airlines|
|        airlines_csv|
|    airlines_iceberg|
|airlines_iceberg_...|
|      airlines_mjain|
|          airquality|
|                ajvp|
|          atlas_demo|
|            bankdemo|
|          bca_jps_l0|
|     bri_ranger_demo|
|cde_demo_pauldefusco|
|   cde_demo_pdefusco|
|        cde_workshop|
|cde_workshop_pdef...|
| cde_workshop_smohan|
|             cdedemo|
+--------------------+
only showing top 20 rows



In [7]:
training_df = spark.createDataFrame(
[
    ("0", "a b c d e spark", 1.0),
    ("1", "b d", 0.0),
    ("2", "spark f g h", 1.0),
    ("3", "hadoop mapreduce", 0.0),
],
["id", "text", "label"],
)

In [8]:
def exp1(df):

    mlflow.set_experiment("sparkml-experiment")

    ##EXPERIMENT 1

    df.writeTo("spark_catalog.default.training").using("iceberg").createOrReplace()
    spark.sql("SELECT * FROM spark_catalog.default.training").show()

    ### SHOW TABLE HISTORY AND SNAPSHOTS
    spark.read.format("iceberg").load("spark_catalog.default.training.history").show(20, False)
    spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").show(20, False)

    snapshot_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("snapshot_id").tail(1)[0][0]
    committed_at = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("committed_at").tail(1)[0][0].strftime('%m/%d/%Y')
    parent_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("parent_id").tail(1)[0][0]
    
    tags = {
      "iceberg_snapshot_id": snapshot_id,
      "iceberg_snapshot_committed_at": committed_at,
      "iceberg_parent_id": parent_id,
      "row_count": training_df.count()
    }
    
    ### MLFLOW EXPERIMENT RUN
    with mlflow.start_run() as run:

        maxIter=8
        regParam=0.01

        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
        lr = LogisticRegression(maxIter=maxIter, regParam=regParam)
        pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
        model = pipeline.fit(training_df)

        mlflow.log_param("maxIter", maxIter)
        mlflow.log_param("regParam", regParam)

        #prediction = model.transform(test)
        mlflow.set_tags(tags)

    mlflow.end_run()
    
    experiment_id = mlflow.get_experiment_by_name("sparkml-experiment").experiment_id
    runs_df = mlflow.search_runs(experiment_id, run_view_type=1)
    
    return runs_df

In [9]:
def exp2(df):
    
    mlflow.set_experiment("sparkml-experiment")
    
    ##EXPERIMENT 2

    ### ICEBERG INSERT DATA - APPEND FROM DATAFRAME

    # PRE-INSERT
    spark.sql("SELECT * FROM spark_catalog.default.training").show()

    temp_df = spark.sql("SELECT * FROM spark_catalog.default.training")
    temp_df.writeTo("spark_catalog.default.training").append()
    df = spark.sql("SELECT * FROM spark_catalog.default.training")

    # PROST-INSERT
    spark.sql("SELECT * FROM spark_catalog.default.training").show()

    spark.read.format("iceberg").load("spark_catalog.default.training.history").show(20, False)
    spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").show(20, False)

    snapshot_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("snapshot_id").tail(1)[0][0]
    committed_at = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("committed_at").tail(1)[0][0].strftime('%m/%d/%Y')
    parent_id = spark.read.format("iceberg").load("spark_catalog.default.training.snapshots").select("parent_id").tail(1)[0][0]
    
    tags = {
      "iceberg_snapshot_id": snapshot_id,
      "iceberg_snapshot_committed_at": committed_at,
      "iceberg_parent_id": parent_id,
      "row_count": df.count()
    }
    
    ### MLFLOW EXPERIMENT RUN
    with mlflow.start_run() as run:

        maxIter=10
        regParam=0.002

        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
        lr = LogisticRegression(maxIter=maxIter, regParam=regParam)
        pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
        model = pipeline.fit(training_df)

        mlflow.log_param("maxIter", maxIter)
        mlflow.log_param("regParam", regParam)

        #prediction = model.transform(test)
        mlflow.set_tags(tags)

    mlflow.end_run()
    
    experiment_id = mlflow.get_experiment_by_name("sparkml-experiment").experiment_id
    runs_df = mlflow.search_runs(experiment_id, run_view_type=1)
    
    return runs_df

In [10]:
def exp3(df, snapshot_id):
    ##EXPERIMENT 3

    df = spark.read.option("snapshot-id", snapshot_id).table("spark_catalog.default.training")

    committed_at = spark.sql("SELECT committed_at FROM spark_catalog.default.training.snapshots WHERE snapshot_id = {};".format(snapshot_id)).collect()[0][0].strftime('%m/%d/%Y')
    parent_id = str(spark.sql("SELECT parent_id FROM spark_catalog.default.training.snapshots WHERE snapshot_id = {};".format(snapshot_id)).tail(1)[0][0])

    tags = {
      "iceberg_snapshot_id": snapshot_id,
      "iceberg_snapshot_committed_at": committed_at,
      "iceberg_parent_id": parent_id,
      "row_count": training_df.count()
    }

    ### MLFLOW EXPERIMENT RUN
    with mlflow.start_run() as run:

        maxIter=7
        regParam=0.005

        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
        lr = LogisticRegression(maxIter=maxIter, regParam=regParam)
        pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
        model = pipeline.fit(training_df)

        mlflow.log_param("maxIter", maxIter)
        mlflow.log_param("regParam", regParam)

        #prediction = model.transform(test)
        mlflow.set_tags(tags)

    mlflow.end_run()
    
    experiment_id = mlflow.get_experiment_by_name("sparkml-experiment").experiment_id
    runs_df = mlflow.search_runs(experiment_id, run_view_type=1)

    #spark.stop()
    
    return runs_df

In [11]:
exp1(training_df)

2023/11/05 19:47:41 INFO mlflow.tracking.fluent: Experiment with name 'sparkml-experiment' does not exist. Creating a new experiment.
23/11/05 19:47:41 WARN HiveMetaStoreClient: Failed to connect to the MetaStore Server...
23/11/05 19:47:58 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+

+-----------------------+------------------+---------+-------------------+
|made_current_at        |snapshot_id       |parent_id|is_current_ancestor|
+-----------------------+------------------+---------+-------------------+
|2023-11-05 19:48:13.242|426957395241265414|null     |true               |
+-----------------------+------------------+---------+-------------------+

+-----------------------+-------------------+-------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------

23/11/05 19:48:23 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/05 19:48:23 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/11/05 19:48:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/11/05 19:48:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.regParam,params.maxIter,tags.iceberg_snapshot_id,tags.iceberg_snapshot_committed_at,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.mlflow.source.name,tags.mlflow.user,tags.iceberg_parent_id,tags.row_count,tags.engineID
0,toi7-h745-fb0i-o9pg,pbq3-oss4-0wox-z4x4,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/pbq3-oss4-0wox-z4x4/to...,2023-11-05 19:48:18.394889216+00:00,2023-11-05 19:48:24.708999936+00:00,0.01,8,426957395241265414,11/05/2023,LOCAL,06b7bed0031ad636f6b3ade4189ea15b164906f3,/usr/local/lib/python3.9/site-packages/ipykern...,pauldefusco,None,4,3fw9f74t2egfv60p


In [12]:
exp2(training_df)

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+

+-----------------------+-------------------+------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id         |is_current_ancestor|
+-----------------------+-------------------+------------------+-------------------+
|2023-11-05 19:48:13.242|426957395241265414 |null              |true               |
|2023-11-05 19:51:28.961|4225021400360977669|426957395241265414|true               |
+-----------------------+-------------------+------------------+-------------------+

+-----------------------+-------------------+-------------------+---------+-----------------------------------------------------------------

,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.regParam,params.maxIter,tags.iceberg_snapshot_id,tags.iceberg_snapshot_committed_at,tags.mlflow.source.type,tags.mlflow.source.git.commit,tags.mlflow.source.name,tags.mlflow.user,tags.iceberg_parent_id,tags.row_count,tags.engineID
0,toi7-h745-fb0i-o9pg,pbq3-oss4-0wox-z4x4,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/pbq3-oss4-0wox-z4x4/to...,2023-11-05 19:48:18.394889216+00:00,2023-11-05 19:48:24.708999936+00:00,0.01,8,426957395241265414,11/05/2023,LOCAL,06b7bed0031ad636f6b3ade4189ea15b164906f3,/usr/local/lib/python3.9/site-packages/ipykern...,pauldefusco,None,4,3fw9f74t2egfv60p
1,bokv-w00w-q6zu-zvsy,pbq3-oss4-0wox-z4x4,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/pbq3-oss4-0wox-z4x4/bo...,2023-11-05 19:51:31.007222016+00:00,2023-11-05 19:51:36.945999872+00:00,0.002,10,4225021400360977669,11/05/2023,LOCAL,06b7bed0031ad636f6b3ade4189ea15b164906f3,/usr/local/lib/python3.9/site-packages/ipykern...,pauldefusco,426957395241265414,8,3fw9f74t2egfv60p


In [22]:
#Retrieve snapshot_id from Experiments page or above dataframe. Use the Snapshot ID from the first experiment.
snapshot_id = "5150219139107687331"
exp3(training_df, snapshot_id)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,params.maxIter,params.regParam,tags.mlflow.source.git.commit,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.source.type,tags.row_count,tags.iceberg_parent_id,tags.iceberg_snapshot_id,tags.iceberg_snapshot_committed_at
0,kfnc-iesm-r79x-nqcp,p1y7-kv7u-fkya-ca0e,EXPERIMENT_RUN_RUNNING,/home/cdsw/.experiments/p1y7-kv7u-fkya-ca0e/kf...,2023-03-29 01:04:44.571115008+00:00,NaT,10,0.001,1d14407660f321086ef27b91b0e06ca91cd34b18,/usr/local/lib/python3.7/site-packages/ipykern...,pauldefusco,LOCAL,None,None,None,None
1,vaza-u8gd-dbkd-k4ow,p1y7-kv7u-fkya-ca0e,EXPERIMENT_RUN_FAILED,/home/cdsw/.experiments/p1y7-kv7u-fkya-ca0e/va...,2023-03-29 01:06:53.296070144+00:00,2023-03-29 01:06:56.807000064+00:00,8,0.01,1d14407660f321086ef27b91b0e06ca91cd34b18,/usr/local/lib/python3.7/site-packages/ipykern...,pauldefusco,LOCAL,None,None,None,None
2,55oy-jgoj-sxwp-tp7b,p1y7-kv7u-fkya-ca0e,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/p1y7-kv7u-fkya-ca0e/55...,2023-03-29 01:09:43.688317184+00:00,2023-03-29 01:09:46.864000+00:00,8,0.01,1d14407660f321086ef27b91b0e06ca91cd34b18,/usr/local/lib/python3.7/site-packages/ipykern...,pauldefusco,LOCAL,4,None,5150219139107687331,03/29/2023
3,bmxd-ywal-739v-3ikq,p1y7-kv7u-fkya-ca0e,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/p1y7-kv7u-fkya-ca0e/bm...,2023-03-29 01:11:43.908041984+00:00,2023-03-29 01:11:48.220000+00:00,10,0.002,1d14407660f321086ef27b91b0e06ca91cd34b18,/usr/local/lib/python3.7/site-packages/ipykern...,pauldefusco,LOCAL,8,5150219139107687331,8171776268264279750,03/29/2023
4,tndc-jcty-7qn9-jxsu,p1y7-kv7u-fkya-ca0e,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/p1y7-kv7u-fkya-ca0e/tn...,2023-03-29 01:14:32.279436032+00:00,2023-03-29 01:14:35.438000128+00:00,7,0.005,1d14407660f321086ef27b91b0e06ca91cd34b18,/usr/local/lib/python3.7/site-packages/ipykern...,pauldefusco,LOCAL,4,5150219139107687331,8171776268264279750,03/29/2023
5,4yk0-cwcg-9hcn-c7k9,p1y7-kv7u-fkya-ca0e,EXPERIMENT_RUN_FINISHED,/home/cdsw/.experiments/p1y7-kv7u-fkya-ca0e/4y...,2023-03-29 01:15:29.313084928+00:00,2023-03-29 01:15:32.180000+00:00,7,0.005,1d14407660f321086ef27b91b0e06ca91cd34b18,/usr/local/lib/python3.7/site-packages/ipykern...,pauldefusco,LOCAL,4,None,5150219139107687331,03/29/2023
